# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

S. Li  ->  S. Li  |  ['S. Li']
T. Henning  ->  T. Henning  |  ['T. Henning']
D. Semenov  ->  D. Semenov  |  ['D. Semenov']
Arxiv has 70 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2410.18801
extracting tarball to tmp_2410.18801... done.
Retrieving document from  https://arxiv.org/e-print/2410.18941


extracting tarball to tmp_2410.18941...

 done.


T. Henning  ->  T. Henning  |  ['T. Henning']
D. Semenov  ->  D. Semenov  |  ['D. Semenov']


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure overview_L1448N
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure overview_L1448N as tmp_2410.18941/./overview_L1448N.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure continuum_L1448N_li
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure continuum_L1448N_li as tmp_2410.18941/./continuum_L1448N_li.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))


Found 100 bibliographic references in tmp_2410.18941/L1448N_PRODIGE_v9.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.18941-b31b1b.svg)](https://arxiv.org/abs/2410.18941) | **PRODIGE -- envelope to disk with NOEMA. IV. An infalling gas bridge surrounding two Class 0/I systems in L1448N**  |
|| C. Gieser, et al. -- incl., <mark>T. Henning</mark>, <mark>D. Semenov</mark> |
|*Appeared on*| *2024-10-25*|
|*Comments*| *17 pages, 10 figures*|
|**Abstract**|            Context. The formation of stars has been subject to extensive studies in the past decades from molecular cloud to protoplanetary disk scales. It is still not fully understood how the surrounding material in a protostellar system, that often shows asymmetric structures with complex kinematic properties, feeds the central protostar(s) and their disk(s). Aims. We study the spatial morphology and kinematic properties of the molecular gas surrounding the IRS3A and IRS3B protostellar systems in the L1448N region located in the Perseus molecular cloud. Methods. We present 1 mm NOEMA observations of the PRODIGE large program and analyze the kinematic properties of molecular lines. Given the complexity of the spectral profiles, the lines are fitted with up to three Gaussian velocity components. The clustering algorithm DBSCAN is used to disentangle the velocity components into the underlying physical structure. Results. We discover an extended gas bridge (~3000 au) surrounding both the IRS3A and IRS3B systems in six molecular line tracers (C18O, SO, DCN, H2CO, HC3N, and CH3OH). This gas bridge is oriented along the northeast-southwest direction and shows clear velocity gradients on the order of 100 km/s/pc towards the IRS3A system. We find that the observed velocity profile is consistent with analytical streamline models of gravitational infall towards IRS3A. The high-velocity C18O (2-1) emission towards IRS3A indicates a protostellar mass of ~1.2 Msun. Conclusions. While high angular resolution continuum data often show IRS3A and IRS3B in isolation, molecular gas observations reveal that these systems are still embedded within a large-scale mass reservoir with a complex spatial morphology as well as velocity profiles. The kinematic properties of the extended gas bridge are consistent with gravitational infall toward the IRS3A protostar.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2410.18801-b31b1b.svg)](https://arxiv.org/abs/2410.18801) | **Enhanced Peak and Extended Cooling of the Extreme-ultraviolet Late Phase in a Confined Solar Flare**  |
|| <mark>S. Li</mark>, et al. |
|*Appeared on*| *2024-10-25*|
|*Comments*| *17 pages, 10 figures, accepted for publication in ApJ*|
|**Abstract**|            We present observations and analysis of an X1.8 non-eruptive solar flare on 2012 October 23, which is characterized by an extremely large late-phase peak seen in the warm coronal extreme-ultraviolet (EUV) emissions ($\sim$ 3 MK), with the peak intensity over 1.4 times that of main flare peak. The flare is driven by a failed eruption of a magnetic flux rope (MFR), whose strong squeeze force acting on the overlying magnetic structures gives rise to an intense early heating of the late-phase loops. Based on differential emission measure (DEM) analysis, it is found that the late-phase loops experience a "longer-than-expected" cooling without the presence of any obvious additional heating, and meanwhile, their volume emission measure (EM) maintains a plateau level for a long time before turning into an evident decay. Without the need for an additional heating, we propose that the special thermodynamic evolution of the late-phase loops revealed in this flare might arise from loop cross-sectional expansions with height, which are evidenced by both direct measurements from EUV images and by magnetic field extrapolation. By blocking the losses of both heat flux and mass from the corona, such an upward cross-sectional expansion not only elongates the loop cooling time, but also more effectively sustains the loop density, therefore leading to a later-than-expected occurrence of the warm coronal late phase in combination with a sufficiently high late-phase peak. We further verify such a scenario by analytically solving the cooling process of a late-phase loop characterized by a variable cross section.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2410.18941/./overview_L1448N.png', 'tmp_2410.18941/./continuum_L1448N_li.png', 'tmp_2410.18941/./moment0_L1448N_all.png', 'tmp_2410.18941/./L1448N_outflows.png', 'tmp_2410.18941/./L1448N_outflows_spectra.png']
copying  tmp_2410.18941/./overview_L1448N.png to _build/html/
copying  tmp_2410.18941/./continuum_L1448N_li.png to _build/html/
copying  tmp_2410.18941/./moment0_L1448N_all.png to _build/html/
copying  tmp_2410.18941/./L1448N_outflows.png to _build/html/
copying  tmp_2410.18941/./L1448N_outflows_spectra.png to _build/html/
exported in  _build/html/2410.18941.md
    + _build/html/tmp_2410.18941/./overview_L1448N.png
    + _build/html/tmp_2410.18941/./continuum_L1448N_li.png
    + _build/html/tmp_2410.18941/./moment0_L1448N_all.png
    + _build/html/tmp_2410.18941/./L1448N_outflows.png
    + _build/html/tmp_2410.18941/./L1448N_outflows_spectra.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\arraystretch}{1.1}$
$\newcommand{\arraystretch}{1.1}$
$\newcommand\as{\ifmmode{\rlap.} " \! \else {\rlap.} " \!\fi}$</div>



<div id="title">

# PRODIGE - envelope to disk with NOEMA.: IV. An infalling gas bridge surrounding two Class 0/I systems in L1448N

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2410.18941-b31b1b.svg)](https://arxiv.org/abs/2410.18941)<mark>Appeared on: 2024-10-25</mark> -  _17 pages, 10 figures_

</div>
<div id="authors">

C. Gieser, et al. -- incl., <mark>T. Henning</mark>, <mark>D. Semenov</mark>

</div>
<div id="abstract">

**Abstract:** The formation of stars has been subject to extensive studies in the past decades from molecular cloud to protoplanetary disk scales. It is still not fully understood how the surrounding material in a protostellar system, that often shows asymmetric structures with complex kinematic properties, feeds the central protostar(s) and their disk(s). We study the spatial morphology and kinematic properties of the molecular gas surrounding the IRS3A and IRS3B protostellar systems in the L1448N region located in the Perseus molecular cloud. We present 1 mm NOEMA observations of the PRODIGE large program and analyze the kinematic properties of molecular lines. Given the complexity of the spectral profiles, the lines are fitted with up to three Gaussian velocity components. The clustering algorithm \texttt{DBSCAN} is used to disentangle the velocity components into the underlying physical structure. We discover an extended gas bridge ( $\approx$ 3000 au) surrounding both the IRS3A and IRS3B systems in six molecular line tracers (C $^{18}$ O, SO, DCN, H $_{2}$ CO, HC $_{3}$ N, and CH $_{3}$ OH). This gas bridge is oriented along the northeast-southwest direction and shows clear velocity gradients on the order of 100 km s $^{-1}$ pc $^{-1}$ towards the IRS3A system. We find that the observed velocity profile is consistent with analytical streamline models of gravitational infall towards IRS3A. The high-velocity C $^{18}$ O ( $2-1$ ) emission towards IRS3A indicates a protostellar mass of $\sim$ 1.2 $M_\odot$ . While high angular resolution continuum data often show IRS3A and IRS3B in isolation, molecular gas observations reveal that these systems are still embedded within a large-scale mass reservoir with a complex spatial morphology as well as velocity profiles. The kinematic properties of the extended gas bridge are consistent with gravitational infall toward the IRS3A protostar.

</div>

<div id="div_fig1">

<img src="tmp_2410.18941/./overview_L1448N.png" alt="Fig6.1" width="50%"/><img src="tmp_2410.18941/./continuum_L1448N_li.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** Continuum images toward L1448N. The left panel shows in color the JCMT 850 $\upmu$m emission taken from the COMPLETE survey \citep[][]{Ridge2006, Kirk2006}. The grey dashed circles are the primary beam (22$\farcs$8) of the two NOEMA pointings of the PRODIGE observations. The protostellar systems of L1448N (IRS3A, IRS3B, and IRS3C), as well as the nearby L1448-mm system are marked by black squares. The beam of JCMT and NOEMA observations is shown in the bottom right corner in green and black, respectively. In the right panel the 1.4 mm continuum image of the PRODIGE observations is presented in color and white contours. Contour levels are 5, 10, 20, 40, 80, 160$\times \sigma_\mathrm{cont}$($\sigma_\mathrm{cont}$=0.94 mJy beam$^{-1}$). The black circles mark the positions of individual protostars taken from the VANDAM survey  ([Tobin, Looney and Li 2016]()) . The synthesized beam of the NOEMA data is shown in the bottom left corner. In both panels scale bars are marked in the top left corner and bipolar outflow orientations are highlighted by red and blue arrows (Sect. \ref{sec:outflow}). (*fig:continuum*)

</div>
<div id="div_fig2">

<img src="tmp_2410.18941/./moment0_L1448N_all.png" alt="Fig7" width="100%"/>

**Figure 7. -** Line integrated intensity maps of the PRODIGE observations. The line integrated intensity and the 1.4 mm continuum is presented in color and white contours, respectively. Contour levels are 5, 40, 160$\times \sigma_\mathrm{cont}$($\sigma_\mathrm{cont}$=0.94 mJy beam$^{-1}$). The IRS3A and IRS3B protostellar systems toward L1448N are labeled in white. The black circles mark the positions of individual protostars taken from the VANDAM survey  ([Tobin, Looney and Li 2016]()) . Bipolar outflow orientations are indicated by blue and red arrows. The synthesized beam of the line and continuum data is indicated in the bottom left and bottom right corner, respectively. A scale bar is shown in the top right corner. In the bottom row panels, the extent of bridge structure is indicated by the dashed pink polygon. Spectra toward the continuum peak position of IRS3A and IRS3B are shown in Fig. \ref{fig:avg_spectra} for all transitions. (*fig:moment0*)

</div>
<div id="div_fig3">

<img src="tmp_2410.18941/./L1448N_outflows.png" alt="Fig8.1" width="50%"/><img src="tmp_2410.18941/./L1448N_outflows_spectra.png" alt="Fig8.2" width="50%"/>

**Figure 8. -** Molecular outflows in L1448N. In the top panels, the red and blue contours show the red- and blueshifted line integrated intensities of low-velocity CO $2-1$(left), high-velocity CO $2-1$(center), and SiO $5-4$(right). Contour levels are 0.1, 0.3, 0.5, 0.9$\times$ the corresponding peak integrated intensity. In color the integrated intensity around the source velocity is presented. The integration ranges are listed in each panel and indicated by red and blue dashed vertical lines in the spectra in the bottom panels. Outflow orientations are indicated by blue and red arrows. The black contours are the 1.4 mm continuum of the PRODIGE observations. Contour levels are 5, 10, 20, 40, 80, 160$\times \sigma_\mathrm{cont}$($\sigma_\mathrm{cont}$=0.94 mJy beam$^{-1}$). The protostellar systems of the L1448N system (IRS3A and IRS3B) are labeled in black. The synthesized beam of the line and continuum data is indicated in the bottom left and bottom right corner, respectively. A scale bar is shown in the top right corner. The bottom panels show spectra extracted from the positions indicated by the black triangles in the top panels. The grey vertical dashed line is the region velocity of L1448N ($\approx$5 km s$^{-1}$). (*fig:outflow*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2410.18941"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

229  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
